The key thing I'll work towards here is a `Model` that has a dictionary of `params` and a `forward` method.

First, the `Tensor` class. 

There are two concepts here: there is a Tensor that takes in Tensors as inputs and produces Tensors as outputs, and a Tensor that instead produces a _number_ as an output.

The first we'll call a `Node`, and the second we'll call a `Loss`.

In [1]:
import inspect
from typing import List, NamedTuple, Callable, Optional, Union, Iterator

import numpy as np

In [2]:
Arrayable = Union[float, list, np.ndarray]

Tensorable = Union['Tensor', float, np.ndarray]

def ensure_tensor(tensorable: Tensorable) -> 'Tensor':
    if isinstance(tensorable, Tensor):
        return tensorable
    else:
        return Tensor(tensorable)
    
def ensure_array(arrayable: Arrayable) -> np.ndarray:
    if isinstance(arrayable, np.ndarray):
        return arrayable
    else:
        return np.array(arrayable)

class Dependency(NamedTuple):
    tensor: 'Tensor'
    grad_fn: Callable[[np.ndarray], np.ndarray]
        
def collapse_sum(grad: np.ndarray,
                 t: 'Tensor') -> np.ndarray:

    # Sum out added dims
    ndims_added = grad.ndim - t.data.ndim
    for _ in range(ndims_added):
        grad = grad.sum(axis=0)

    # Sum across broadcasted (but non-added dims)
    for i, dim in enumerate(t.shape):
        if dim == 1:
            grad = grad.sum(axis=i, keepdims=True)
    
    return grad

In [3]:
class Tensor:
    
    def __init__(self,
                 data: np.ndarray,
                 depends_on: List[Dependency] = None,
                 no_grad: bool = False) -> None:
        self.data = ensure_array(data)
        self.depends_on = depends_on or []
        self.no_grad = no_grad
        self.shape = self.data.shape
        self.grad: Optional['Tensor'] = None
        if not self.no_grad:
            self.zero_grad()

    def __repr__(self) -> str:
        return f"Tensor({self.data})"
            
    def __add__(self, other: Tensorable) -> 'Tensor':
        """gets called if I do t + other"""
        return _add(self, ensure_tensor(other))

    def __radd__(self, other: Tensorable) -> 'Tensor':
        """gets called if I do other + t"""
        return _add(ensure_tensor(other), self)

    def __iadd__(self, other: Tensorable) -> 'Tensor':
        """when we do t += other"""
        self.data = self.data + ensure_tensor(other).data
        return self    

    def __isub__(self, other: Tensorable) -> 'Tensor':
        """when we do t -= other"""
        self.data = self.data - ensure_tensor(other).data
        return self
    
    def __imul__(self, other: Tensorable) -> 'Tensor':
        """when we do t *= other"""
        self.data = self.data * ensure_tensor(other).data
        return self

    def __mul__(self, other: Tensorable) -> 'Tensor':
        return _mul(self, ensure_tensor(other))

    def __rmul__(self, other: Tensorable) -> 'Tensor':
        return _mul(ensure_tensor(other), self)

    def __matmul__(self, other: Tensorable) -> 'Tensor':
        return _matmul(self, other)

    def __neg__(self) -> 'Tensor':
        return _neg(self)

    def __sub__(self, other: Tensorable) -> 'Tensor':
        return _sub(self, ensure_tensor(other))

    def __rsub__(self, other: Tensorable) -> 'Tensor':
        return _sub(ensure_tensor(other), self)
    
    def __getitem__(self, idxs) -> 'Tensor':
        return _slice(self, idxs)
    
    def zero_grad(self) -> None:
        self.grad = Tensor(np.zeros_like(self.data, dtype=np.float64),
                           no_grad = True)
    
    def sum(self) -> 'Tensor':
        return tensor_sum(self)
    
    def backward(self, grad: 'Tensor' = None) -> None:
        # backward mostly going to be called on the loss after calling "sum"
        if self.no_grad:
            return
        
        if self.shape == ():
            grad = Tensor(np.array(1.0))

        
        self.grad.data = self.grad.data + grad.data

        for dependency in self.depends_on:
            backward_grad = dependency.grad_fn(grad.data)
            dependency.tensor.backward(Tensor(backward_grad))

In [4]:
def _add(t1: Tensor, t2: Tensor) -> Tensor:

    def _forward(t1: Tensor, t2: Tensor):
        return t1.data + t2.data

    def t1_grad(grad: np.ndarray):

        grad = collapse_sum(grad, t1)

        return grad

    def t2_grad(grad: np.ndarray):

        grad = collapse_sum(grad, t2)

        return grad

    data = _forward(t1, t2)
    depends_on = [
        Dependency(t1, t1_grad),
        Dependency(t2, t2_grad)
    ]
    return Tensor(data, depends_on)

def _mul(t1: Tensor, t2: Tensor) -> Tensor:

    def _forward(t1: Tensor, t2: Tensor):
        return t1.data * t2.data

    def t1_grad(grad: np.ndarray):
        grad = grad * t2.data
        grad = collapse_sum(grad, t1)

        return grad

    def t2_grad(grad: np.ndarray):
        grad = grad * t1.data
        grad = collapse_sum(grad, t2)

        return grad

    data = _forward(t1, t2)
    depends_on = [
        Dependency(t1, t1_grad),
        Dependency(t2, t2_grad)
    ]
    return Tensor(data, depends_on)

def _matmul(t1: Tensor, t2: Tensor) -> Tensor:

    assert t1.shape[1] == t2.shape[0]

    def _forward(t1: Tensor, t2: Tensor):
        return t1.data @ t2.data

    def t1_grad(grad: np.ndarray):
        grad = grad @ t2.data.T

        return grad

    def t2_grad(grad: np.ndarray):
        grad = t1.data.T @ grad

        return grad

    data = _forward(t1, t2)
    depends_on = [
        Dependency(t1, t1_grad),
        Dependency(t2, t2_grad)
    ]
    return Tensor(data, depends_on)

def _neg(t: Tensor) -> Tensor:

    def _forward(t: Tensor):
        return -t.data

    def t_grad(grad: np.ndarray):
        return -grad

    data = _forward(t)
    depends_on = [
        Dependency(t, t_grad),
    ]
    return Tensor(data, depends_on)

def _sub(t1: Tensor, t2: Tensor) -> Tensor:
    return t1 + -t2

def _slice(t: Tensor, idxs: slice) -> Tensor:

    def _forward(t: Tensor):
        return t.data[idxs]

    data = _forward(t)    
    
    def t_grad(grad: np.ndarray) -> np.ndarray:
        bigger_grad = np.zeros_like(data)
        try:
            bigger_grad[idxs] = grad
        except:
            import pdb; pdb.set_trace()
        return bigger_grad

    depends_on = [
        Dependency(t, t_grad),
    ]

    return Tensor(data, depends_on)

def tensor_sum(t: Tensor) -> Tensor:

    def _forward(t: Tensor):
        return t.data.sum()

    def t_grad(grad: np.ndarray) -> np.ndarray:
        return grad * np.ones_like(t.data)

    data = _forward(t)
    depends_on = [
        Dependency(t, t_grad),
    ]

    return Tensor(data, depends_on)        
        

In [5]:
class Parameter(Tensor):
    def __init__(self, *shape) -> None:
        data = np.random.randn(*shape)
        super().__init__(data)

In [6]:
class Model:
    def parameters(self) -> Iterator[Parameter]:
        for name, value in inspect.getmembers(self):
            if isinstance(value, Parameter):
                yield value
            elif isinstance(value, Model):
                yield from value.parameters()

    def zero_grad(self):
        for parameter in self.parameters():
            parameter.zero_grad()

In [7]:
class SGD:
    def __init__(self, lr: float = 0.01) -> None:
        self.lr = lr

    def step(self, model: Model) -> None:
        for parameter in model.parameters():
            parameter -= parameter.grad * self.lr


In [8]:
def tanh(t: Tensor) -> Tensor:
    def _forward(t: Tensor):
        return np.tanh(t.data)

    data = _forward(t)
    
    def t_grad(grad: np.ndarray) -> np.ndarray:
        return grad * (1 - data * data)

    depends_on = [
        Dependency(t, t_grad)
    ]
    
    return Tensor(data, depends_on)

### Boston data

In [9]:
from sklearn.datasets import load_boston

boston = load_boston()

data = boston.data
target = boston.target
features = boston.feature_names

from sklearn.preprocessing import StandardScaler
s = StandardScaler()
data = s.fit_transform(data)

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=80718)

y_train, y_test = y_train.reshape((-1,1)), y_test.reshape((-1,1))

X_train, X_test, y_train, y_test = (Tensor(X_train, no_grad=True),
                                    Tensor(X_test, no_grad=True),
                                    Tensor(y_train, no_grad=True),
                                    Tensor(y_test, no_grad=True))

In [10]:
class BostonModel(Model):
    def __init__(self, num_hidden: int = 13) -> None:
        self.w1 = Parameter(13, num_hidden)
        self.b1 = Parameter(num_hidden)

        self.w2 = Parameter(num_hidden, 1)
        self.b2 = Parameter(1)

    def predict(self, inputs: Tensor) -> Tensor:
        # inputs will be (batch_size, 13)
        x1 = inputs @ self.w1 + self.b1  # (batch_size, num_hidden)
        x2 = tanh(x1)                    # (batch_size, num_hidden)
        x3 = x2 @ self.w2 + self.b2      # (batch_size, 4)

        return x3

### Train model

In [12]:
optimizer = SGD(lr=0.0001)
batch_size = 32
model = BostonModel()
train_size = X_train.shape[0]

for epoch in range(10):
    epoch_loss = 0.0

    for start in range(0, train_size, batch_size):
        end = start + batch_size

        model.zero_grad()

        inputs = X_train[start:end]
        inputs.no_grad = True

        predicted = model.predict(inputs)
        actual = y_train[start:end]
        actual.no_grad = True
        errors = predicted - actual
        loss = (errors * errors).sum()
#         import pdb; pdb.set_trace()
        loss.backward()
        optimizer.step(model)
        
        
        # test predictions
    predicted = model.predict(X_test)
    errors = predicted - y_test
    loss = (errors * errors).sum()
        
    print(epoch, loss)

0 Tensor(62876.462152720575)
1 Tensor(43271.918829249815)
2 Tensor(29745.67211898419)
3 Tensor(20476.232541083977)
4 Tensor(14789.277680349456)
5 Tensor(11691.356242168664)
6 Tensor(9993.953047534596)
7 Tensor(9008.462128137302)
8 Tensor(8370.862590280502)
9 Tensor(7904.851412313703)
